In [1]:
import numpy as np
import sympy as sp
import pandas as pd
import copy

In [25]:
SIZE = 4
presicion = 4
epsilon = 10**(-presicion+1)
epsilon

0.001

In [26]:
def get_indexes_max_nondiag(Matrix):
    indexes = (1, 0)
    for i in range(Matrix.shape[0]):
        for j in range(i):
            indexes = (i, j) if abs(Matrix[indexes]) < abs(Matrix[i, j]) else indexes
    return indexes


def get_max_diag(Matrix):
    max_value = 0
    for i in range(Matrix.shape[0]):
        max_value = abs(Matrix[i, i]) if max_value < abs(Matrix[i, i]) else max_value
    return max_value


def sgn(value):
    return 1 if value >= 0 else -1


def validation_defination(Matrix, sigma_elem):
    obstacle = np.sqrt(get_max_diag(Matrix)) * sigma_elem
    for i in range(Matrix.shape[0]):
        for j in range(Matrix.shape[1]):
            if (i!=j and abs(Matrix[i, j]) > obstacle):
                return True
    return False

    
def rotations_with_obstacles(Matrix, p):
    sigmas = [10**(-k) for k in range(p)]
    W = sp.eye(Matrix.shape[0])
    
    counter = 0
    
    for sigma_elem in sigmas:
        while (validation_defination(Matrix, sigma_elem)):
            
            counter+=1
            
            row_max, col_max = get_indexes_max_nondiag(Matrix)
            
            d = np.sqrt((Matrix[row_max, row_max] - Matrix[col_max, col_max])**2 + 4*(Matrix[row_max, col_max]**2))
            c = np.sqrt((1+(abs(Matrix[row_max, row_max] - Matrix[col_max, col_max]) / d)) / 2)
            s = sgn(Matrix[row_max, col_max] * (Matrix[row_max, row_max] - Matrix[col_max, col_max])) *\
            np.sqrt((1-(abs(Matrix[row_max, row_max] - Matrix[col_max, col_max]) / d)) / 2)
            
            C_matrix = copy.deepcopy(Matrix)
            
            for k in range(Matrix.shape[0]):
                for l in range(Matrix.shape[1]):
                    if k != row_max and k != col_max and l != row_max and l != col_max:
                        None
                    elif k != row_max and k != col_max:
                        C_matrix[k, row_max] = C_matrix[row_max, k] = c * Matrix[k, row_max] + s * Matrix[k, col_max]
                        C_matrix[k, col_max] = C_matrix[col_max, k] =  - s * Matrix[k, row_max] + c * Matrix[k, col_max]
            C_matrix[row_max, row_max] = c**2 * Matrix[row_max, row_max] + c*s*2*Matrix[row_max, col_max] + s**2 * Matrix[col_max, col_max]
            C_matrix[col_max, col_max] = s**2 * Matrix[row_max, row_max] - c*s*2*Matrix[row_max, col_max] + c**2 * Matrix[col_max, col_max]
            C_matrix[row_max, col_max] = C_matrix[col_max, row_max] = 0
            
            T = sp.eye(Matrix.shape[0])
            T[row_max, row_max] = c
            T[row_max, col_max] = -s
            T[col_max, row_max] = s
            T[col_max, col_max] = c
            
            W *= T
            
            Matrix = C_matrix
            
    
    lambdas = [Matrix[i, i] for i in range(Matrix.shape[0])]
    
    
    return Matrix, lambdas, np.matrix([[round(W[i, j], 8) for j in range(SIZE)] for i in range(SIZE)])


def Richardson_method(lambda_min, lambda_max, A, B, epsilon):
    n = 1
    ksi = lambda_min / lambda_max

    while True:
        ro = (1 - np.sqrt(ksi)) / (1 + np.sqrt(ksi))
        q_n = (2*ro**n) / (1 + ro**(2*n))

        if (q_n < epsilon): break
        else: n+=1

    print("Число итераций: n =", n, "\nВеличина сходимости: q_n =", q_n)
    
    ro_0 = (1 - ksi) / (1 + ksi)
    tau_0 = 2 / (lambda_min + lambda_max)
    x_prev = x_new = np.matrix(np.zeros(SIZE)).T

    df_arr = np.array(x_new).flatten()

    for k in range(0, n+1):
        mu_k = np.cos((2*k - 1) * np.pi / (2 * n))
        tau_k = tau_0 / (1 + ro_0 * mu_k)

        x_new = x_prev - tau_k * (A * x_prev - B)

        df_arr = np.vstack([df_arr, np.array(x_new).flatten()])

        x_prev = x_new

    df = pd.DataFrame(df_arr)
    df.columns = [f'X_{i+1}' for i in range(SIZE)]
    return x_new, df

In [27]:
A = np.matrix([
    [10.9, 1.2, 2.1, 0.9],
    [1.2, 11.2, 1.5, 2.5],
    [2.1, 1.5, 9.8, 1.3],
    [0.9, 2.5, 1.3, 12.1]
], dtype=np.float64)

A = np.matrix([
    [2, 1],
    [1, 2],
], dtype=np.float64)

B = np.matrix([4, 5], dtype=np.float64).T

SIZE = A.shape[0]

np.array(np.linalg.solve(A, B)).flatten()

array([1., 2.])

In [28]:
Result_Matrix, eigenvalues, vectors = rotations_with_obstacles(A, presicion)

eigenvalues = np.array(eigenvalues)
eigenvalues.sort()
eigenvalues

array([1., 3.])

In [33]:
print("Имеет ли смысл применять метод Ричардсона?")

check = False

if (eigenvalues[0] > 0):
    check = True
    print("Да\n")
else:
    print("Нет")
    
if (check):
    x, table = Richardson_method(eigenvalues[0], eigenvalues[-1], A, B, epsilon)
    print("\nОтвет:", np.array(x).flatten())

Имеет ли смысл применять метод Ричардсона?
Да

Число итераций: n = 6 
Величина сходимости: q_n = 0.0007401924500370105

Ответ: [1.00025807 1.99976744]


In [34]:
table

,X_1,X_2
0,0.000000,0.000000
1,1.348651,1.685814
2,1.219479,1.780125
3,1.138550,1.861493
4,1.077194,1.922792
5,1.032868,1.967143
6,1.007438,1.992549
7,1.000258,1.999767


In [35]:
error1 = sp.Matrix(A * x - B)
print("Погрешность с системой:")
error1

Погрешность с системой:


Matrix([
[0.000283580333354294],
[-0.00020704672179761]])

In [36]:
print("Максимум погрешности с системой:")
max(abs(error1))

Максимум погрешности с системой:


0.000283580333354294

In [37]:
print("Погрешность с линейным алгоритмом numpy:")
error2 = sp.Matrix(np.linalg.solve(A, B) - x)
error2

Погрешность с линейным алгоритмом numpy:


Matrix([
[-0.000258069129502214],
[ 0.000232557925650134]])

In [38]:
print("Максимум погрешности с линейным алгоритмом numpy:")
max(abs(error2))

Максимум погрешности с линейным алгоритмом numpy:


0.000258069129502214